In [1]:
import numpy as np
from scipy.optimize import linprog

In [2]:
from docplex.mp.model import Model
import time
import numpy as np
def create_bpp(weights,bin_size):
    model = Model('BinPacking')
    n = len(weights)  # number of items
    m = n  # number of bins(worst case each item occupies one  bin)
    #Decision variable
    # x[i,j] = 1 if i occupies bin j
    x= model.binary_var_matrix(n,m,name='x', key_format='item_{0}_bin_{1}')
    # y[j]= 1 is bin j is used
    y=model.binary_var_list(m,name='y', key_format='bin_{0}') 
    # Objective: Minimize number of bins used
    model.minimize(model.sum(y[j] for j in range(m)))
    #Constraints:
    for i in range(n):
        item_assign=0
        for j in range(m):
            item_assign+=x[i,j]
    model.add_constraint(item_assign==1,f'item {i} assigned')

    for j in range(m):
        bin_capacity=0
        for i in range(n):
           bin_capacity+=weights[i]* x[i,j]
        model.add_constraint(bin_capacity<=bin_size*y[j],f'bin {j} capacity')
    
    return model, x, y   

    

In [3]:
import numpy as np

def ilp_to_qubo(model, x, y, weights, bin_size, A, B, C):
  
    n = len(weights)  # number of items
    m = n  # maximum number of bins
    Q = np.zeros((n*m + m, n*m + m))
    
    # Create variable to index mapping
    var_to_index = {}
    for i in range(n):
        for j in range(m):
            var_to_index[f'item_{i}_bin_{j}'] = i * m + j
    for j in range(m):
        var_to_index[f'bin_{j}'] = n * m + j
    
    # Process constraints from the model
    for constraint in model.iter_constraints():
        # Get left and right expressions
        left_expr = constraint.get_left_expr()
        right_expr = constraint.get_right_expr()
        sense=constraint.sense
        # Get the terms from the left expression
        terms = []
        for term in left_expr.iter_terms():
            var, coef = term[0], term[1]  # Unpack term tuple correctly
            var_name = var.get_name()
            if var_name in var_to_index:
                idx = var_to_index[var_name]
                terms.append((idx, coef))
        
       # Add quadratic penalties based on constraint type
        if sense == '==':  # equality constraint (one bin per item)
            # (sum x_ij - 1)^2 penalty
            for idx1, coef1 in terms:
                Q[idx1, idx1] += B
                for idx2, coef2 in terms:
                    if idx1 < idx2:
                        Q[idx1, idx2] += 2 * B
                        Q[idx2, idx1] += 2 * B
            
            # Add constant term to complete (sum x_ij - 1)^2
            for idx1, _ in terms:
                Q[idx1, idx1] -= 2 * B
        
        elif sense == '<=':  # inequality constraint (bin capacity)
            rhs = float(right_expr.constant if hasattr(right_expr, 'constant') else right_expr)
            
            # Add capacity constraint penalties
            for idx1, coef1 in terms:
                for idx2, coef2 in terms:
                    Q[idx1, idx2] += C * coef1 * coef2
                    
            # Add interaction with bin usage variable
            bin_idx = next((idx for idx, coef in terms if 'bin_' in var_to_index.keys()[idx]), None)
            if bin_idx is not None:
                for idx, coef in terms:
                    if idx != bin_idx:
                        Q[idx, bin_idx] -= C * coef * rhs
                        Q[bin_idx, idx] -= C * coef * rhs
                
                # Add squared term for bin capacity
                Q[bin_idx, bin_idx] += C * rhs * rhs
    
    # Add objective function (minimize number of bins)
    for j in range(m):
        Q[n*m + j, n*m + j] += A
        
    # Add additional penalties to ensure items are assigned
    for i in range(n):
        penalty = 1000  # Large penalty for not assigning items
        row_sum = 0
        for j in range(m):
            idx = var_to_index[f'item_{i}_bin_{j}']
            row_sum += 1
            Q[idx, idx] += penalty
        
        # Subtract penalty for correct assignment
        for j in range(m):
            idx = var_to_index[f'item_{i}_bin_{j}']
            Q[idx, idx] -= 2 * penalty * row_sum
            for k in range(j+1, m):
                idx2 = var_to_index[f'item_{i}_bin_{k}']
                Q[idx, idx2] += 2 * penalty
                Q[idx2, idx] += 2 * penalty
    
    return Q

In [4]:
def qubo(model,x,y,weights, bin_size, A, B, C):
    n = len(weights)  # number of items
    m = n  # maximum number of bins (worst case: one item per bin)
    Q= np.zeros((n*m+m, n*m+m))
    # Get constraints from the CPLEX model
    constraints = model.iter_constraints()
    for j in range(m):  #adding penalty coefficient for using a bin
        Q[n*m+j,n*m+j]+=A
    for i in range(n): #to ensure that each item belongs to exactly one bin
        for j in range(m):
            Q[i*m+j,i*m+j]+=B #add a penalty for self assignment
            for k in range(j+1,m): #penalty for pair of bins
                Q[i*m+j,i*m+k]+= 2*B# double the penalty for multiple assignments
        Q[i*m+m-1,i*m+m-1]-=B#to adjust the last bin penalty
    for i in range(m): #penalty for exceeding bin capacity
        for j in range(n):
            for k in range (j,n):
                Q[j*m+i, k*m+j]+= 2*C*weights[j]*weights[k]  #k*m+i
            for i in range(n):
                Q[i*m+j,n*m+j]-= 2*C*weights[i]*bin_size
            Q[n*m+j, n*m+j]+=C*bin_size**2
            
    return Q    
  

In [5]:
def qubo_to_ising(Q):
    #converting QUBO matrix to ising hamiltonian
    n= Q.shape[0]
    h=np.zeros(n) #linear terms
    J=np.zeros((n,n)) #quadratic terms(couplings)
    constant= 0 #Constant term 
    for i in range(n):  
        for j in range(i,n):
            if i==j:
                h[i]+=Q[i,j]/2 #linear terms(left diagonal terms of qubo)
                constant += Q[i, i] / 4  # Extra factor for constant shift
            else:
                J[i,j]=Q[i,j]/4#symmetric distribution of off diagonal elements of qubo
                J[j,i]=Q[i,j]/4
                constant += Q[i, j] / 4  # Constant shift
    return h,J,constant

In [6]:
import pennylane as qml
from pennylane import numpy as np
def ising_hamil(h,J,constant):
    n_q=len(h)
    coeff=[]
    ops=[]
    #Add coonstant term as an identity operator
    coeff.append(constant)
    ops.append(qml.Identity(0))#acts on dummy qubit 

   #single term: h[]*Z
    for i in range(n_q):
       if abs(h[i])>0:
           coeff.append(h[i])
           ops.append(qml.PauliZ(i))
   #two-qubit interaction terms: J[]*Z*Z
    for i in range (n_q):
       for j in range (i+1,n_q):
           if abs(J[i,j])>0:
               coeff.append(J[i,j])
               ops.append(qml.PauliZ(i)@qml.PauliZ(j))
   #define the hamiltonian
    ising_h= qml.Hamiltonian(coeff,ops) 
    return ising_h
               
    
    

In [7]:
import pennylane as qml
import pennylane as qml
import numpy as np

def create_ansatz(H, nq, p):
    """Creates a QAOA ansatz circuit with p layers for nq qubits using Hamiltonian H."""
    dev = qml.device('default.qubit', wires=nq)

    @qml.qnode(dev)
    def qaoa_circuit(params):
        # Split parameters into gamma (cost) and beta (mixer) parameters
        gammas = params[:p]
        betas = params[p:]
        
        # Initial state: apply Hadamard gates to all qubits for superposition
        for j in range(nq):
            qml.Hadamard(wires=j)
        
        for layer in range(p):
            # Cost layer using Hamiltonian H
            # Iterate through operators and coefficients separately
            for op, coeff in zip(H.ops, H.coeffs):
                # Skip identity term
                if isinstance(op, qml.Identity):
                    continue
                    
                if isinstance(op, qml.PauliZ):
                    # Single qubit term (Z)
                    wire = op.wires[0]
                    qml.RZ(2 * gammas[layer] * coeff, wires=wire)
                elif isinstance(op, qml.operation.Operation) and len(op.wires) == 2:
                    # Two-qubit term (ZZ)
                    wire1, wire2 = op.wires
                    qml.CNOT(wires=[wire1, wire2])
                    qml.RZ(2 * gammas[layer] * coeff, wires=wire2)
                    qml.CNOT(wires=[wire1, wire2])

            # Mixer layer: X rotations for each qubit
            for j in range(nq):
                qml.RX(2 * betas[layer], wires=j)
                
        # Return expectation value of the Hamiltonian
        return qml.expval(H)
    # Initial parameter values
    #init_params = np.random.uniform(0, np.pi, 2 * p)
    
    # Return the expectation value
    return qaoa_circuit

In [10]:
import pennylane as qml
import numpy as np
import time

def run_qaoa(h, J, constant, p=2):
    """Runs the QAOA with Pennylane and returns optimal value, parameters, and execution time."""
    nq = len(J)  # number of qubits
     
    # Step 1: Create the QAOA ansatz
    qaoa_circuit = create_ansatz(h,nq, p)
    np.random.seed(42)
    # Initialize trainable parameters
    init_params = np.random.uniform(0, np.pi, 2 * p)
    
    # Make parameters trainable
    params = qml.numpy.array(init_params, requires_grad=True)
    optimizer = qml.AdamOptimizer(stepsize=0.1)
    
    # Start timing
    start_time = time.time()
    
    # Store optimization history
    energy_history = []
    
    # Run optimization
    for _ in range(50):
        params, energy = optimizer.step_and_cost(qaoa_circuit, params)
        energy_history.append(energy)
    
    exec_time = time.time() - start_time
    
    # Get final energy
    final_energy = qaoa_circuit(params)
    
    return final_energy, params, exec_time

def solve(weights_list, bin_size):
    model, x, y = create_bpp(weights_list, bin_size)
    Q = ilp_to_qubo(model, x, y, weights_list, bin_size, A=0.01, B=1.0, C=1.0)
    h, J, constant = qubo_to_ising(Q)
    H= ising_hamil(h,J,constant)
    nq = len(h)
    
    optimal_value, optimal_params, exec_time = run_qaoa(H, J, constant, p=4)

    return {
        'problem_size': len(weights_list),
        'optimal_value': float(optimal_value),
        'execution': exec_time,
        'parameters': optimal_params.tolist()
    }  

def instance():
    small = np.random.randint(1, 10, size=2)
    medium = np.random.randint(1, 20, size=4)
    large = np.random.randint(1, 50, size=7)
    bin_size = 15

    print("Running QAOA on small instance:")
    result_1 = solve(small, bin_size)
    print(f"Small instance results: {result_1}")
    
    print("Running QAOA on medium instance:")
    result_2 = solve(medium, bin_size)
    print(f"Medium instance results: {result_2}")
    
    
    # print("Running QAOA on large instance:")
    #result_3 = solve(large, bin_size)
    # print(f"Large instance results: {result_3}")
    
    results = {
        'small': result_1,
        'medium': result_2,
         #'large': result_3  # Uncomment if needed
    }
    print("Final Results:", results)

# Run instance to get all results
instance()



Running QAOA on small instance:
Small instance results: {'problem_size': 2, 'optimal_value': -2823.3072676410507, 'execution': 1.7204806804656982, 'parameters': [1.343022292548519, 3.73871320188552, 4.287511721508329, 4.125808118342571, -0.4964539226053633, 0.7792015493917053, 1.0208794112475723, 1.1960418783286746]}
Running QAOA on medium instance:
Medium instance results: {'problem_size': 4, 'optimal_value': -29046.99693084616, 'execution': 510.90227937698364, 'parameters': [1.765205961876392, 2.893111344048057, 1.9045660080108686, 2.34044792338807, 2.043366687854126, -0.014813863207445457, -1.1429863074338706, 2.93144049778697]}
Final Results: {'small': {'problem_size': 2, 'optimal_value': -2823.3072676410507, 'execution': 1.7204806804656982, 'parameters': [1.343022292548519, 3.73871320188552, 4.287511721508329, 4.125808118342571, -0.4964539226053633, 0.7792015493917053, 1.0208794112475723, 1.1960418783286746]}, 'medium': {'problem_size': 4, 'optimal_value': -29046.99693084616, 'exe